In [1]:
import os
import sys
import time
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
from datetime import datetime
from model_settings import ms
from feature_collector import preprocess_data
from df_collector import df_collector
root = Path().resolve().parent.parent
df_collector.root = root

In [2]:
from convsklearn import barrier_trainer, asian_trainer
price_name = 'barrier_price' 
filetag = 'cboe barriers'
trainer = barrier_trainer

dataset = df_collector.cboe_spx_barriers().iloc[:,1:]
dataset = dataset[dataset[price_name]<dataset['spot_price']].dropna().reset_index(drop=True).copy()
dates = pd.Series(np.sort(dataset['date'].unique()))

In [3]:
dataset

,spot_price,strike_price,barrier,days_to_maturity,updown,outin,w,barrier_type_name,rebate,dividend_rate,risk_free_rate,theta,kappa,rho,eta,v0,calculation_date,date,barrier_price
0,4271.75,3844.575,2135.875,60.0,Down,Out,call,DownOut,0.0,0.0,0.04,0.127234,0.792037,-0.646272,1.054022,0.032848,2022-08-17 15:52:15.386,2022-08-18,479.950906
1,4271.75,3844.575,2135.875,60.0,Down,Out,put,DownOut,0.0,0.0,0.04,0.127234,0.792037,-0.646272,1.054022,0.032848,2022-08-17 15:52:15.386,2022-08-18,27.172207
2,4271.75,3844.575,2135.875,60.0,Down,In,call,DownIn,0.0,0.0,0.04,0.127234,0.792037,-0.646272,1.054022,0.032848,2022-08-17 15:52:15.386,2022-08-18,0.253186
3,4271.75,3844.575,2135.875,60.0,Down,In,put,DownIn,0.0,0.0,0.04,0.127234,0.792037,-0.646272,1.054022,0.032848,2022-08-17 15:52:15.386,2022-08-18,0.378806
4,4271.75,3844.575,2135.875,90.0,Down,Out,call,DownOut,0.0,0.0,0.04,0.127234,0.792037,-0.646272,1.054022,0.032848,2022-08-17 15:52:15.386,2022-08-18,509.339939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21689455,5813.50,6394.850,8720.250,540.0,Up,In,put,UpIn,0.0,0.0,0.04,0.087499,1.982234,-0.657095,1.705566,0.018946,2024-10-15 16:50:04.131,2024-10-16,6.577341
21689456,5813.50,6394.850,8720.250,720.0,Up,Out,call,UpOut,0.0,0.0,0.04,0.087499,1.982234,-0.657095,1.705566,0.018946,2024-10-15 16:50:04.131,2024-10-16,377.391470
21689457,5813.50,6394.850,8720.250,720.0,Up,Out,put,UpOut,0.0,0.0,0.04,0.087499,1.982234,-0.657095,1.705566,0.018946,2024-10-15 16:50:04.131,2024-10-16,650.459798
21689458,5813.50,6394.850,8720.250,720.0,Up,In,call,UpIn,0.0,0.0,0.04,0.087499,1.982234,-0.657095,1.705566,0.018946,2024-10-15 16:50:04.131,2024-10-16,189.459253


In [4]:
dataset['moneyness'] = ms.df_moneyness(dataset)
dataset['m'] = dataset['moneyness']/dataset['spot_price']
dataset['b'] = dataset['barrier']/dataset['spot_price']

In [5]:
train_dates = dates.iloc[:len(dates)//4]
test_dates = dates[~dates.isin(train_dates)]

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.neural_network import MLPRegressor

In [7]:
feature_set = [
    'm',
    'b',
    'days_to_maturity',
    'risk_free_rate',
    'dividend_rate',
    'kappa',
    'theta',
    'rho',
    'eta',
    'v0',
    'barrier',
    'barrier_type_name',
    'w'
]
feature_set

['m',
 'b',
 'days_to_maturity',
 'risk_free_rate',
 'dividend_rate',
 'kappa',
 'theta',
 'rho',
 'eta',
 'v0',
 'barrier',
 'barrier_type_name',
 'w']

In [8]:
preprocessor = ColumnTransformer(transformers=[
    ('categorical',OneHotEncoder(),['w','averaging_type'])
],remainder='passthrough',force_int_remainder_cols=False)

dnn_pipeline = make_pipeline(preprocessor,MLPRegressor(random_state=1312))
dnn = dnn_pipeline.fit(train_X,train_y)

NameError: name 'train_X' is not defined

In [ ]:
train_data = dataset[dataset['date'].isin(train_dates)]
train_X = train_data[feature_set]
train_y = train_data['observed_price']

In [ ]:
test_data = dataset[dataset['date'].isin(test_dates)]
test_X = test_data[feature_set]
train_y = test_data['observed_price']
np.mean(np.abs(dnn.predict(test_X)-train_y))